In [1]:
#import
from pyspark.sql import SparkSession
import os
import glob

# Initialize SparkSession
spark = SparkSession.builder.appName("TPC-DS Data Loading").config("spark.sql.catalogImplementation", "hive").getOrCreate()

23/10/22 14:50:11 WARN Utils: Your hostname, alex-Lenovo-Legion-5-15IMH05H resolves to a loopback address: 127.0.1.1; using 192.168.1.40 instead (on interface wlp0s20f3)
23/10/22 14:50:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/22 14:50:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.catalog.listTables()

23/10/22 14:50:13 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/10/22 14:50:13 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/10/22 14:50:18 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
23/10/22 14:50:18 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore alex@127.0.1.1
23/10/22 14:50:18 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
23/10/22 14:50:19 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


[]

In [3]:
#create all tables
with open(os.getcwd() + "/Create_database.sql", 'r') as file: # path/to/tpcds.sql
    db_string = file.read()

db_string = str.split(db_string, ";")
db_string = db_string[:-1]

for string in db_string:
    if string != '\n' or string != '':
        spark.sql(string)

23/10/22 14:50:20 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
23/10/22 14:50:20 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
23/10/22 14:50:20 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
23/10/22 14:50:20 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/10/22 14:50:20 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/10/22 14:50:20 WARN HiveMetaStore: Location: file:/home/alex/Projet%20DW/DataWarehouse-SparkSQL/spark-warehouse/call_center specified for non-external table:call_center
23/10/22 14:50:20 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can se

In [4]:
#load datas into tables
data_file = glob.glob(os.getcwd() + "/data/*")
for i in data_file:
    table_name = i.split("/")[-1].split(".")[0]
    if table_name != "dbgen_version":
        sql_command = "select * from " + table_name
        tmp=spark.sql(sql_command)
        #print(tmp)
        df = spark.read.schema(tmp.schema).csv( i, sep='|')
        df.write.mode("append").insertInto(table_name)
        print("Data from table" + table_name + "is inserted")

In [5]:
#run queries
queries_files = glob.glob(os.getcwd() + "/queries/queries*")
for i in queries_files:
    with open( i , 'r') as file:
        db_string = file.read()
        
        #start time
        try:
            spark.sql(db_string)
        except:
            print( i + "doesn't work")
        #end time
    


IndentationError: expected an indented block after 'with' statement on line 4 (79727229.py, line 5)

In [6]:
spark.sql("select count(*) from customer").show()


+--------+
|count(1)|
+--------+
|       0|
+--------+

